In [1]:
## building an auto-encoder class
## builiding the encoder

6

In [ ]:
from keras import Model 
from keras.layers import Input, Conv2D

class Autoencoder:
    
    def __init__(self,
                 input_shape,
                 conv_filters,
                 conv_kernels,
                 conv_strides,
                 latent_space_dim):
        self.input_shape = input_shape # [28,28,1] this means the input is a 28x28 image with 1 channel (grayscale) like mnist
        self.conv_filters = conv_filters #[2,4,8] this means the first layer has two filters, the second 4, and the last 8.
        self.conv_kernels = conv_kernels #[3,5,3] this means the first layer has a 3x3 kernel, the second 5x5, and the last 3x3.
        self.conv_strides= conv_strides  #[1,2,2] this means the first layer has a stride of 1, the second 2, and the last 2.
        self.latent_space_dim = latent_space_dim # 2 this means the bottleneck will only have 2 dimensions.
        self.encoder= None
        self.decoder = None
        self.model= None
        self._num_conv_layers= len(conv_filters)
        self._build()
    def _build(self):
        self._build_encoder()
        self._build_decoder()
        self._build_autoencoder()
    def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers= self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self.encoder = Model(encoder_input, bottleneck, name='encoder')
    
    def _add_encoder_input(self):
        return Input(shape=self.input_shape, name='encoder_input')
        
    ## This will create all convolutional blocks in encoder    
    def _add_conv_layers(self, encoder_input):
        x= encoder_input
        for layer_index in range(self._num_conv_layers):
            x = self._add_conv_layer(layer_index, x)
        return x   
    
    def _add_conv_layer(self, layer_index, x):
        layer_number= layer_index+1
        ## Adds a convolutional block to a graph of layers, consisting of a conv 2d+ batch norm + relu 
        conv_layer = Conv2D(filters=self.conv_filters[layer_index],kernel_size=self.conv_kernels[layer_index],
                            strides=self.conv_strides[layer_index], 
                            padding='same', name=f'encoder_conv_layer_{layer_number}')
    